In [1]:
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [2]:
# (Código para instalação de bibliotecas e configuração da API - já presente no seu notebook)

import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [8]:
from google import genai
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"  # Ou outro modelo adequado

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import warnings
warnings.filterwarnings("ignore")

# Função para chamar o agente
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response
# Função para formatar a saída como Markdown
def to_markdown(text):
    text = text.replace('•', '* ')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: text))

In [9]:
# --- Agente 1: Buscador de Notícias ---
def agente_buscador_fraudes(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador_fraudes",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa especializado em golpes e fraudes.
        A sua tarefa é usar a ferramenta de busca do Google (google_search)
        para recuperar as últimas notícias, informações e dicas sobre o tópico abaixo.
        Foque em identificar:
        - Os tipos de golpes e fraudes mais comuns e recentes.
        - Os métodos utilizados pelos golpistas.
        - As formas de prevenção e proteção contra esses golpes.
        - Notícias relevantes sobre golpes que tenham ocorrido recentemente.
        Limite-se aos 5 resultados mais relevantes e confiáveis.
        """,
        description="Agente que busca informações sobre golpes e fraudes no Google",
        tools=[google_search],
    )

    entrada_do_agente_planejador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    informacoes = call_agent(buscador, entrada_do_agente_planejador)
    return informacoes

In [10]:
# --- Agente 2: Educador ---
def agente_educador_fraudes(topico, informacoes_buscadas):
    educador = Agent(
        name="agente_educador_fraudes",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em educação sobre golpes e fraudes online e offline.
        Com base nas informações fornecidas pelo agente de busca, sua tarefa é criar um plano detalhado para uma série de posts educativos em redes sociais.

        O plano deve incluir:

        1.  Tema Central: Defina o tema central da série de posts (por exemplo, "Golpes do Pix", "Phishing", "Fraudes em Investimentos").
        2.  Tópicos Específicos: Divida o tema central em tópicos específicos e relevantes para cada post individual.
            Exemplos de tópicos:
            -   Como identificar um golpe.
            -   Os sinais de alerta de uma fraude.
            -   Dicas práticas para se proteger.
            -   O que fazer se você for vítima de um golpe.
            -   Exemplos de casos reais (se aplicável).
        3.  Público-Alvo: Considere que o público-alvo principal são pessoas de todas as idades, com ênfase em idosos, que podem ser mais vulneráveis a golpes.
        4.  Tom de Voz: Utilize um tom de voz claro, objetivo, informativo e empático. Evite jargões técnicos e adapte a linguagem para ser acessível a todos.
        5.  Formato dos Posts: Sugira formatos adequados para diferentes redes sociais (por exemplo, posts curtos para o Twitter, posts mais longos para o Facebook, listas e infográficos para o Instagram).
        6.  Chamada para Ação: Inclua sugestões de chamadas para ação para incentivar o engajamento do público (por exemplo, "Compartilhe com seus amigos", "Deixe suas dúvidas nos comentários", "Acesse o link na bio para mais informações").
        7.  Uso de Recursos Visuais: Sugira o uso de recursos visuais (imagens, vídeos, infográficos) para tornar os posts mais atraentes e informativos.

        Você pode usar a ferramenta de busca do Google (google_search) para obter mais informações sobre os golpes mencionados e encontrar exemplos ou estatísticas relevantes.

        Ao final, retorne o plano detalhado, incluindo o tema central, os tópicos específicos, sugestões de formato e chamadas para ação.
        """,
        description="Agente que planeja conteúdo educativo sobre golpes e fraudes",
        tools=[google_search]
    )

    entrada_do_agente_educador = f"Tópico: {topico}\nInformações buscadas: {informacoes_buscadas}"
    plano_do_post = call_agent(educador, entrada_do_agente_educador)
    return plano_do_post

In [11]:
# --- Agente 3: Redator de Posts para Redes Sociais ---
def agente_redator_redes_sociais(topico, plano_de_post):
    redator = Agent(
        name="agente_redator_redes_sociais",
        model="gemini-2.0-flash",  # Ou um modelo mais adequado para geração de texto
        instruction="""
            Você é um redator criativo e persuasivo, especializado em criar posts informativos e engajadores sobre golpes e fraudes para diversas redes sociais.
            Com base no plano de post fornecido, sua tarefa é gerar posts escritos adaptados para cada plataforma.

            O plano de post conterá o tema central, os tópicos específicos e sugestões de formato para cada rede social.
            Você deve seguir as seguintes diretrizes:

            1.  Tom de Voz: Utilize um tom de voz claro, conciso, informativo e amigável. Adapte o tom para ser adequado ao público de cada plataforma (por exemplo, mais informal no Twitter, mais explicativo no Facebook).
            2.  Linguagem: Use linguagem acessível, evitando jargões técnicos. Explique conceitos complexos de forma simples e fácil de entender.
            3.  Engajamento: Crie posts que incentivem a interação do público (perguntas, chamadas para ação, etc.).
            4.  Formato: Siga as sugestões de formato do plano de post para cada rede social:
                -   Twitter: Posts curtos e concisos (máximo de 280 caracteres), com foco em informações essenciais e chamadas para ação. Use hashtags relevantes.
                -   Facebook: Posts mais longos e explicativos, com detalhes sobre o golpe, dicas de prevenção e links para fontes confiáveis.
                -   Instagram: Legendas informativas e envolventes para posts visuais (imagens ou vídeos). Use emojis e hashtags relevantes.
                -   Outras Redes: Se o plano incluir outras redes, adapte o formato e o estilo de acordo.
            5.  Precisão: Garanta que as informações fornecidas sejam precisas e atualizadas. Consulte fontes confiáveis se necessário.
            6.  Chamada para Ação: Inclua chamadas para ação claras e relevantes (por exemplo, "Compartilhe com seus amigos", "Acesse o link na bio para saber mais", "Deixe suas dúvidas nos comentários").

            Gere um post para cada rede social especificada no plano.
            """,
        description="Agente redator de posts informativos e engajadores sobre golpes e fraudes para redes sociais",
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    posts_redes_sociais = call_agent(redator, entrada_do_agente_redator)
    return posts_redes_sociais

In [12]:
# --- Agente 4: Revisor de Conteúdo sobre Golpes e Fraudes ---
def agente_revisor_fraudes(topico, posts_redes_sociais):
    revisor = Agent(
        name="agente_revisor_fraudes",
        model="gemini-2.0-flash",
        instruction="""
            Você é um revisor especializado em conteúdo educativo sobre golpes e fraudes para redes sociais.
            Sua tarefa é revisar os posts gerados para diferentes plataformas, garantindo que sejam precisos, claros, concisos, envolventes e adequados para o público, com ênfase na proteção contra golpes.

            Para cada post, verifique os seguintes aspectos:

            1.  Precisão das Informações: As informações sobre o golpe ou fraude estão corretas e atualizadas? Há alguma informação enganosa ou imprecisa?
            2.  Clareza e Concisão: A linguagem utilizada é clara e fácil de entender para um público amplo, incluindo idosos? As frases são concisas e diretas?
            3.  Engajamento: O post é envolvente e tem potencial para gerar interação (compartilhamentos, comentários, etc.)? Há alguma chamada para ação clara?
            4.  Adequação à Plataforma: O formato e o estilo do post são adequados para a rede social em questão (Twitter, Facebook, Instagram, etc.)? O uso de hashtags e emojis é apropriado?
            5.  Tom de Voz: O tom de voz é informativo, empático e adequado para alertar sobre os perigos de golpes e fraudes?
            6.  Prevenção e Proteção: O post destaca claramente as formas de prevenção e proteção contra o golpe ou fraude em questão? As dicas são práticas e acionáveis?
            7.  Impacto Educacional: O post cumpre seu objetivo de educar e conscientizar o público sobre os riscos de golpes e fraudes?

            Para cada post revisado, forneça um feedback detalhado, indicando os pontos fortes e sugerindo melhorias específicas para aumentar a clareza, a precisão e o impacto educativo.

            Retorne os posts revisados com suas sugestões.
            """,
        description="Agente revisor de conteúdo educativo sobre golpes e fraudes para redes sociais",
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nPosts para revisão: {posts_redes_sociais}"
    posts_revisados = call_agent(revisor, entrada_do_agente_revisor)
    return posts_revisados

def criar_conteudo_educativo_fraudes():
    data_de_hoje = date.today().strftime("%d/%m/%Y")

    print("🚨 Iniciando o Sistema de Criação de Conteúdo Educativo sobre Golpes e Fraudes 🚨")

    # --- Obter o Tópico do Usuário ---
    topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar conteúdo educativo (ex: 'Golpe do Pix', 'Phishing', 'Fraudes em Investimentos'): ")

    if not topico:
        print("Você esqueceu de digitar o tópico!")
        return  # Encerra a função se o tópico estiver vazio
    else:
        print(f"Maravilha! Vamos criar conteúdo educativo sobre: {topico}")

    # --- Chamar os Agentes em Sequência ---

    # Agente 1: Buscador de Fraudes
    informacoes_buscadas = agente_buscador_fraudes(topico, data_de_hoje) # Passando topico e data para o agente buscador
    print("\n--- Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(informacoes_buscadas))  # Exibe a saída formatada
    print("---------------------------------------------\n")

    # Agente 2: Educador de Fraudes
    plano_de_post = agente_educador_fraudes(topico, informacoes_buscadas)
    print("\n--- Resultado do Agente 2 (Educador) ---\n")
    display(to_markdown(plano_de_post))
    print("---------------------------------------------\n")

    # Agente 3: Redator de Redes Sociais
    posts_redes_sociais = agente_redator_redes_sociais(topico, plano_de_post)
    print("\n--- Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(posts_redes_sociais))
    print("---------------------------------------------\n")

    # Agente 4: Revisor de Conteúdo sobre Fraudes
    posts_revisados = agente_revisor_fraudes(topico, posts_redes_sociais)
    print("\n--- Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(posts_revisados))
    print("---------------------------------------------\n")

    print("✅ Conteúdo educativo sobre golpes e fraudes gerado com sucesso!")

# Chamar a função principal para executar o sistema
criar_conteudo_educativo_fraudes()

🚨 Iniciando o Sistema de Criação de Conteúdo Educativo sobre Golpes e Fraudes 🚨
❓ Por favor, digite o TÓPICO sobre o qual você quer criar conteúdo educativo (ex: 'Golpe do Pix', 'Phishing', 'Fraudes em Investimentos'): como previnir ligaçoes indesejadas 
Maravilha! Vamos criar conteúdo educativo sobre: como previnir ligaçoes indesejadas 

--- Resultado do Agente 1 (Buscador) ---



> Olá! Para te ajudar a prevenir ligações indesejadas, realizei algumas pesquisas no Google. Veja o que encontrei:
> 
> 
> Para prevenir ligações indesejadas de telemarketing e spam, você pode seguir as seguintes dicas e utilizar as ferramentas disponíveis:
> 
> 1.  **Cadastro no "Não Me Perturbe" da Anatel**:
>     *   Acesse o site do Não Me Perturbe.
>     *   Cadastre-se na plataforma, informando seus dados como nome, sobrenome e CPF.
>     *   Escolha o método de cadastro (e-mail, número de telefone ou telefone fixo) e crie uma senha.
>     *   Após a confirmação do cadastro, você pode bloquear as empresas de telemarketing que deseja evitar.
>     *   O bloqueio é efetivado em até 30 dias.
>     *   A plataforma é uma iniciativa de autorregulação das operadoras para melhorar a relação com os consumidores.
> 2.  **Cadastro no Procon**:
>     *   Acesse a plataforma do Procon do seu estado para verificar a disponibilidade do serviço.
>     *   Preencha um cadastro simples com informações como nome, RG, CPF e endereço.
>     *   Em alguns estados, como Minas Gerais e Santa Catarina, o Procon oferece um sistema de bloqueio de telemarketing.
>     *   O bloqueio é válido para ligações, e-mail, SMS e mensagens (spam).
> 3.  **Bloqueio direto no celular**:
>     *   **Android**:
>         *   Abra o discador do telefone e clique nos três pontos verticais.
>         *   Selecione "Configurações" e clique em "Números bloqueados".
>         *   Adicione o número que deseja bloquear ou ative a identificação de número de SPAM.
>         *   Outra opção é acessar o histórico de chamadas recebidas e bloquear o número indesejado.
>     *   **iPhone**:
>         *   Instale aplicativos de bloqueio como TrueCaller ou Hiya.
>         *   Acesse os ajustes do iPhone, clique em "Telefone" e habilite o app instalado na opção "Bloqueio e identificação".
> 4.  **Utilização de aplicativos de terceiros**:
>     *   Existem diversos aplicativos que identificam e bloqueiam chamadas de spam, como Truecaller, Hiya, Bloqueador de Chamadas, Call Blocker e Whoscall.
>     *   Esses aplicativos possuem bancos de dados com números de spam e permitem criar listas de bloqueio personalizadas.
>     *   Alguns aplicativos, como o Truecaller, identificam de onde são as chamadas e bloqueiam SMS também.
> 5.  **Denúncia de ligações de spam**:
>     *   Denuncie ligações indesejadas através do Não Me Perturbe e nos canais de atendimento da Anatel.
> 6.  **Outras dicas**:
>     *   Ative o filtro de chamadas de spam nas configurações do seu celular Android.
>     *   Use a função "Silenciar Desconhecidos" no iPhone para evitar chamadas de números desconhecidos.
>     *   Bloqueie manualmente números de spam no seu dispositivo.
>     *   Cadastre seu número em listas de contatos bloqueados.
>     *   Considere obter um novo número de telefone ou um número virtual.
> 
> **Observações Importantes:**
> 
> *   O prefixo 0303 foi adicionado para identificar chamadas de telemarketing. A Anatel ampliou a obrigatoriedade do uso do prefixo 0303 para todas as organizações que realizam mais de 10 mil chamadas diárias a partir de 5 de janeiro de 2025.
> *   Mesmo com o bloqueio, algumas empresas continuam ligando através de diferentes números. Se as ligações persistirem, denuncie as práticas abusivas aos Procons estaduais.
> *   O "Não Me Perturbe" é eficaz para bloquear telemarketing de empresas de telecomunicações e instituições financeiras, mas não bloqueia ligações de outros setores.
> *   A plataforma "Não Me Perturbe" fechou 2024 com quase 12,5 milhões de números cadastrados, indicando que é uma ferramenta popular para evitar chamadas indesejadas.
> *   Se você não quiser mais usar o aplicativo de bloqueio, você pode removê-lo.
> 
> Espero que estas informações te ajudem a reduzir o recebimento de ligações indesejadas!
> 


---------------------------------------------


--- Resultado do Agente 2 (Educador) ---



> Aqui está um plano detalhado para uma série de posts educativos em redes sociais sobre como prevenir ligações indesejadas, com foco em acessibilidade e clareza para todos os públicos, incluindo idosos:
> 
> **Tema Central:** Proteção contra Ligações Indesejadas e Spam
> 
> **Público-Alvo:** Pessoas de todas as idades, com ênfase em idosos e pessoas menos familiarizadas com tecnologia.
> 
> **Tom de Voz:** Claro, objetivo, informativo e empático. Linguagem simples, sem jargões técnicos.
> 
> **Tópicos Específicos e Formatos de Posts:**
> 
> 1.  **Post 1: "Não Me Perturbe": Sua Arma Contra o Telemarketing Abusivo**
>     *   **Tópico:** O que é o "Não Me Perturbe" da Anatel e como ele pode te ajudar a bloquear ligações de telemarketing de empresas de telecomunicações e financeiras.
>     *   **Formato:**
>         *   Facebook/Instagram: Post curto e direto com um infográfico simples explicando o que é o "Não Me Perturbe".
>         *   Twitter: Thread com frases curtas e links diretos para o site.
>     *   **Recursos Visuais:** Infográfico com o logo do "Não Me Perturbe" e ícones representando ligações sendo bloqueadas.
>     *   **Chamada para Ação:** "Acesse o site do 'Não Me Perturbe' e cadastre seu número! Link na bio."
> 
> 2.  **Post 2: Cadastro no "Não Me Perturbe": Um Passo a Passo Simples**
>     *   **Tópico:** Como se cadastrar no "Não Me Perturbe" da Anatel: um guia passo a passo detalhado e fácil de seguir.
>     *   **Formato:**
>         *   Facebook/Instagram: Post com um carrossel de imagens (Instagram) ou um guia visual (Facebook) mostrando cada etapa do cadastro.
>         *   YouTube/TikTok: Vídeo curto (1-2 minutos) com uma demonstração prática do cadastro no site.
>     *   **Recursos Visuais:** Prints de tela do site do "Não Me Perturbe" com setas e destaques indicando onde clicar.
>     *   **Chamada para Ação:** "Compartilhe este guia com seus amigos e familiares que precisam se proteger de ligações indesejadas!"
> 
> 3.  **Post 3: Procon ao seu Lado: Bloqueando Ligações em Nível Estadual**
>     *   **Tópico:** Como verificar se o Procon do seu estado oferece um serviço de bloqueio de telemarketing e como se cadastrar.
>     *   **Formato:**
>         *   Facebook/Instagram: Post informativo com um mapa do Brasil destacando os estados que oferecem o serviço.
>         *   Blog: Artigo detalhado com links para os sites dos Procons de cada estado.
>     *   **Recursos Visuais:** Mapa do Brasil, logos dos Procons estaduais.
>     *   **Chamada para Ação:** "Acesse o site do Procon do seu estado e descubra se você pode bloquear ligações por lá! Deixe um comentário se precisar de ajuda."
> 
> 4.  **Post 4: Bloqueio Direto no Celular: Android e iPhone**
>     *   **Tópico:** Como bloquear números de telefone diretamente no seu celular (Android e iPhone), com instruções claras e fáceis de seguir.
>     *   **Formato:**
>         *   Facebook/Instagram: Dois posts separados, um para Android e outro para iPhone, com imagens ilustrativas.
>         *   YouTube/TikTok: Vídeos curtos demonstrando o processo em cada sistema operacional.
>     *   **Recursos Visuais:** Prints de tela dos celulares Android e iPhone mostrando onde clicar para bloquear números.
>     *   **Chamada para Ação:** "Marque um amigo que precisa aprender a bloquear números no celular!"
> 
> 5.  **Post 5: Apps que Salvam: Identificando e Bloqueando Spams Automaticamente**
>     *   **Tópico:** Apresentação de aplicativos de terceiros que identificam e bloqueiam chamadas de spam, como Truecaller e Hiya.
>     *   **Formato:**
>         *   Facebook/Instagram: Post com um comparativo dos principais aplicativos, destacando seus recursos e benefícios.
>         *   Blog: Review detalhado de cada aplicativo, com links para download.
>     *   **Recursos Visuais:** Logos dos aplicativos, prints de tela mostrando a identificação de chamadas de spam.
>     *   **Chamada para Ação:** "Você usa algum desses aplicativos? Compartilhe sua experiência nos comentários!"
> 
> 6.  **Post 6: Denuncie! Sua Voz Contra o Abuso**
>     *   **Tópico:** Como denunciar ligações de spam através do "Não Me Perturbe" e nos canais de atendimento da Anatel.
>     *   **Formato:**
>         *   Facebook/Instagram: Post curto e direto com os links e telefones para denúncia.
>         *   Twitter: Thread com informações concisas sobre como denunciar.
>     *   **Recursos Visuais:** Ícones representando telefones e megafones.
>     *   **Chamada para Ação:** "Não se cale! Denuncie as ligações de spam e ajude a combater o telemarketing abusivo."
> 
> 7.  **Post 7: Dicas Extras para se Proteger**
>     *   **Tópico:** Outras dicas úteis para evitar ligações indesejadas, como ativar o filtro de chamadas de spam no Android e usar a função "Silenciar Desconhecidos" no iPhone.
>     *   **Formato:**
>         *   Facebook/Instagram: Lista com as dicas em formato de texto ou infográfico.
>     *   **Recursos Visuais:** Ícones representando cada dica (ex: um filtro para o filtro de chamadas, um megafone silenciado para "Silenciar Desconhecidos").
>     *   **Chamada para Ação:** "Qual dessas dicas você já usa? Compartilhe suas próprias dicas nos comentários!"
> 
> 8.  **Post 8: Alerta! Entenda o Prefixo 0303**
>     *   **Tópico:** Explicação sobre o prefixo 0303, que identifica chamadas de telemarketing, e como ele pode te ajudar a identificar e evitar essas ligações.
>     *   **Formato:**
>         *   Facebook/Instagram: Post informativo com um design chamativo destacando o prefixo 0303.
>     *   **Recursos Visuais:** Imagem de um telefone com o prefixo 0303 em destaque.
>     *   **Chamada para Ação:** "Fique atento! Se você vir o prefixo 0303, já sabe que é telemarketing."
> 
> **Observações Importantes:**
> 
> *   **Adaptação para Idosos:** Use fontes grandes e claras, evite cores que dificultem a leitura e explique cada passo de forma detalhada e paciente.
> *   **Inclusão Digital:** Ofereça versões em áudio dos posts para pessoas com dificuldades de leitura.
> *   **Atualização Constante:** Mantenha os posts atualizados com as últimas informações e ferramentas disponíveis para combater o telemarketing abusivo.
> *   **Interação:** Responda a todos os comentários e dúvidas do público, criando um senso de comunidade e confiança.
> 
> Este plano detalhado oferece uma estrutura abrangente para educar o público sobre como se proteger de ligações indesejadas. Ao adaptar os posts para diferentes formatos e públicos, você pode garantir que a informação chegue a todos de forma clara e acessível.


---------------------------------------------


--- Resultado do Agente 3 (Redator) ---



> Ok, aqui estão os posts para cada rede social, baseados no plano fornecido para o Post 1: "Não Me Perturbe": Sua Arma Contra o Telemarketing Abusivo:
> 
> **Post 1: "Não Me Perturbe": Sua Arma Contra o Telemarketing Abusivo**
> 
> *   **Tópico:** O que é o "Não Me Perturbe" da Anatel e como ele pode te ajudar a bloquear ligações de telemarketing de empresas de telecomunicações e financeiras.
> 
> **Twitter:**
> 
> > Cansado de ligações de telemarketing? 😠 Conheça o "Não Me Perturbe" da Anatel! 🛡️ Bloqueie empresas de telefonia e bancos de te importunarem. ➡️ Cadastre-se agora: [Link encurtado para o site] #NãoMePerturbe #Telemarketing #Anatel #BloquearLigações
> 
> **Facebook:**
> 
> >  Você sabia que pode se livrar das ligações indesejadas de telemarketing de empresas de telefonia e bancos? 🤩 A Anatel criou o serviço "Não Me Perturbe", uma ferramenta gratuita que permite bloquear essas empresas de entrarem em contato com você.
> >
> >  Acesse o site, cadastre seu número e diga adeus às chamadas inconvenientes! 😉
> >
> >  [Infográfico simples explicando o que é o "Não Me Perturbe"]
> >
> >  Saiba mais e cadastre-se aqui: [Link para o site "Não Me Perturbe"]
> >
> >  #NãoMePerturbe #Telemarketing #Anatel #BloquearLigações #Proteção #LigaçõesIndesejadas #Dica
> 
> **Instagram:**
> 
> >  [Imagem: Infográfico com o logo do "Não Me Perturbe" e ícones representando ligações sendo bloqueadas]
> >
> >  Chega de ser incomodado por telemarketing! 😤 O "Não Me Perturbe" da Anatel te ajuda a bloquear ligações de empresas de telefonia e financeiras. 🚫
> >
> >  É fácil, gratuito e funciona! ✅
> >
> >  Acesse o link na bio e cadastre seu número! 🔗
> >
> >  #NãoMePerturbe #Telemarketing #Anatel #BloquearLigações #Proteção #LigaçõesIndesejadas #Dica #Spam #Celular #Brasil #Anatel


---------------------------------------------


--- Resultado do Agente 4 (Revisor) ---



> ## Revisão dos Posts: "Não Me Perturbe": Sua Arma Contra o Telemarketing Abusivo
> 
> **Análise Geral:**
> 
> Os posts estão bons e cumprem o objetivo de informar sobre o "Não Me Perturbe". No entanto, podemos otimizar cada um para aumentar o engajamento e a clareza, especialmente para públicos menos familiarizados com o tema (como idosos). É importante garantir que a linguagem seja acessível e que as instruções sejam claras.
> 
> **Twitter:**
> 
> *   **Pontos Fortes:** Conciso e direto, usa emojis para chamar a atenção.
> *   **Sugestões:**
>     *   Adicionar um verbo de ação mais forte, como "Livre-se" ou "Acabe" em vez de "Conheça".
>     *   Substituir "te importunarem" por algo mais direto como "ligarem sem parar".
>     *   Trocar o emoji de raiva (😠) por um mais amigável e menos agressivo, como um de alívio (😌) ou um rosto pensativo (🤔).
>     *   Incluir um call to action ainda mais direto: "Livre-se do telemarketing! Cadastre-se AGORA no 'Não Me Perturbe' da Anatel!".
> *   **Post Revisado:** "Livre-se do telemarketing! 😌 Cadastre-se AGORA no 'Não Me Perturbe' da Anatel! 🛡️ Bloqueie telefonia e bancos de ligarem sem parar. ➡️ [Link] #NãoMePerturbe #Telemarketing #Anatel"
> 
> **Facebook:**
> 
> *   **Pontos Fortes:** Mais detalhado que o post do Twitter, explica o serviço e oferece um infográfico.
> *   **Sugestões:**
>     *   Usar linguagem mais acessível. Em vez de "ferramenta gratuita que permite bloquear essas empresas de entrarem em contato com você", usar algo como "jeito grátis de impedir que essas empresas te liguem".
>     *   O infográfico é crucial. Garantir que ele seja visualmente claro e fácil de entender, com textos curtos e explicativos.
>     *   Adicionar um exemplo de como o serviço "Não Me Perturbe" pode melhorar a vida do usuário (ex: "Imagine ter mais paz e sossego em casa!").
>     *   Incluir um call to action mais direto no meio do texto, além do final.
> *   **Post Revisado:**
>     > Cansado de receber ligações de telemarketing de bancos e operadoras? 🤩 Sabia que a Anatel criou um jeito GRÁTIS de impedir que essas empresas te liguem? É o serviço "Não Me Perturbe"! Imagine ter mais paz e sossego em casa! 😉
>     >
>     > Acesse o site, cadastre seu número e diga adeus às chamadas inconvenientes! ✅ [Link para o site "Não Me Perturbe"]
>     >
>     > [Infográfico simples explicando o que é o "Não Me Perturbe"]
>     >
>     > Saiba mais e cadastre-se aqui: [Link para o site "Não Me Perturbe"]
>     >
>     > #NãoMePerturbe #Telemarketing #Anatel #BloquearLigações #Proteção #LigaçõesIndesejadas #Dica
> 
> **Instagram:**
> 
> *   **Pontos Fortes:** Visualmente atraente (com o infográfico) e usa emojis.
> *   **Sugestões:**
>     *   O texto deve ser mais direto. Em vez de "Chega de ser incomodado por telemarketing!", usar algo como "Se livre das ligações de telemarketing!".
>     *   Reforçar a gratuidade do serviço.
>     *   No Instagram, o link na bio é fundamental. Garantir que ele esteja sempre atualizado.
>     *   Adicionar um call to action mais específico na legenda: "Clique no link da bio e cadastre seu número AGORA para bloquear as ligações!".
> *   **Post Revisado:**
>     > [Imagem: Infográfico com o logo do "Não Me Perturbe" e ícones representando ligações sendo bloqueadas]
>     >
>     > Se livre das ligações de telemarketing! 😤 O "Não Me Perturbe" da Anatel te ajuda a bloquear ligações de empresas de telefonia e financeiras. 🚫 É GRÁTIS e funciona! ✅
>     >
>     > Clique no link da bio e cadastre seu número AGORA para bloquear as ligações! 🔗
>     >
>     > #NãoMePerturbe #Telemarketing #Anatel #BloquearLigações #Proteção #LigaçõesIndesejadas #Dica #Spam #Celular #Brasil #Anatel
> 
> **Considerações Adicionais:**
> 
> *   **Público-Alvo:** Considerar que muitos idosos podem não estar familiarizados com o termo "Anatel".  Em algumas postagens, pode ser útil explicar brevemente o que é a Anatel (Agência Nacional de Telecomunicações) para aumentar a compreensão.
> *   **Testes A/B:** Realizar testes A/B com diferentes versões dos posts para identificar o que gera mais engajamento.
> *   **Monitoramento:** Monitorar os comentários e mensagens para responder a dúvidas e fornecer suporte aos usuários.
> 
> Ao implementar essas sugestões, você pode aumentar a eficácia dos posts e alcançar um público maior, ajudando mais pessoas a se protegerem contra o telemarketing abusivo.
> 


---------------------------------------------

✅ Conteúdo educativo sobre golpes e fraudes gerado com sucesso!
